# Install required libraries and [Contourf and cartopy issues resolved](https://linuxtut.com/en/8f0d69169dc6ab753e08/)

In [1]:
!pip install xarray
!pip install cartopy

!grep '^deb ' /etc/apt/sources.list | \
  sed 's/^deb /deb-src /g' | \
  tee /etc/apt/sources.list.d/deb-src.list
!apt-get -qq update

!grep '^deb ' /etc/apt/sources.list | \
  sed 's/^deb /deb-src /g' | \
  tee /etc/apt/sources.list.d/deb-src.list
!apt-get update

!apt-get -qq build-dep python3-cartopy
!apt-get -qq remove python-shapely python3-shapely

!pip install --no-binary shapely shapely --force
!pip install --no-binary cartopy cartopy==0.19.0

!pip list | grep Shapely

     |████████████████████████████████| 10.8 MB 6.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 49.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 40.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 12.1 MB 50.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 44 kB 2.3 MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516279 sha256=8a1fc54218b08f28c0e39ad48270136bb3a217a92c02c664726cdfee012ac8f5
  Stored in directory: /root/.cache/pip/wheels/98/01/f7/bd10aeb96fe4b518cde5f7c4f5e12c7202f85b7353a5017847
Successfully built cartopy
deb-src h

# Install libraries

In [2]:
import numpy as np  # import numpy package 
import matplotlib.pyplot as plt # function in matplotlib that produces quick and easy plots
import pandas as pd # load the Pandas module, which is a library for data manipulation of 2 dimensional data
import xarray as xr # load the xarray module, which is a library for data manipulation of N dimensional data
import cartopy.crs as ccrs # Cartopy map projections
import cartopy.feature as cfeature # Cartopy map features
from cartopy.util import add_cyclic_point
import cartopy
import warnings 
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from torch.autograd import Variable
# New - may need to install statsmodels and sklearn packages
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error # may need to install sklearn - machine learning package

# Load data

* The data 'download.nc' had been downloaded using the 'data_collection.ipynb' file. I kept it as a separate file, because the website might keep changing the format of the data, time to time. I want the results to be consistent.

In [3]:
# open_dataset is one of the main functions in Xarray to open a Netcdf file.  
data = xr.open_dataset('download.nc') # open the dataset
data 

<xarray.Dataset>
Dimensions:    (latitude: 61, longitude: 71, time: 792)
Coordinates:
  * longitude  (longitude) float32 72.0 72.1 72.2 72.3 ... 78.7 78.8 78.9 79.0
  * latitude   (latitude) float32 22.0 21.9 21.8 21.7 ... 16.3 16.2 16.1 16.0
  * time       (time) datetime64[ns] 2019-07-14 ... 2021-07-24T23:00:00
Data variables:
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    ssr        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-03-23 20:02:03 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

## Sub-data

Since there is a lot of data and many of the locations have null values (sea, ocean). I need to consider the region surrounding Mahad and Mahabaleshwar.

So, I have decided to consider the latitude from 17.4 to 18.33 and longitude from 73.24 to 73.88

In [4]:
lat1 = 18.3
lat2 = 17
lon1 = 73.24
lon2 = 73.88

# find the nearest lat and lon to the given lat and lon
lat1_idx = np.abs(data.latitude.values - lat1).argmin()
lat2_idx = np.abs(data.latitude.values - lat2).argmin()
lon1_idx = np.abs(data.longitude - lon1).argmin()
lon2_idx = np.abs(data.longitude - lon2).argmin()

# get the lat and lon values for the given indices
lat1 = data.latitude[lat1_idx]
lat2 = data.latitude[lat2_idx]
lon1 = data.longitude[lon1_idx]
lon2 = data.longitude[lon2_idx]

# consider the data ranging between lat1 and lat2 and lon1 and lon2
data_subset = data.sel(latitude=slice(lat1, lat2), longitude=slice(lon1, lon2))
data_subset


<xarray.Dataset>
Dimensions:    (latitude: 14, longitude: 8, time: 792)
Coordinates:
  * longitude  (longitude) float32 73.2 73.3 73.4 73.5 73.6 73.7 73.8 73.9
  * latitude   (latitude) float32 18.3 18.2 18.1 18.0 ... 17.3 17.2 17.1 17.0
  * time       (time) datetime64[ns] 2019-07-14 ... 2021-07-24T23:00:00
Data variables:
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    ssr        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-03-23 20:02:03 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

# convert to pandas dataframe

In [5]:
# convert to pandas dataframe
data_df = data_subset.to_dataframe()
data_df.head()



v10         t2m         ssr  \
latitude  longitude time                                                    
18.299999 73.199997 2019-07-14 00:00:00  1.373911  297.332672  12508035.0   
                    2019-07-14 01:00:00  1.345363  297.305328      7625.0   
                    2019-07-14 02:00:00  1.217245  297.981140    161045.0   
                    2019-07-14 03:00:00  1.339882  297.890137    403873.0   
                    2019-07-14 04:00:00  1.227065  298.832306   1164014.0   

                                                   sp        tp  
latitude  longitude time                                         
18.299999 73.199997 2019-07-14 00:00:00  99645.671875  0.010906  
                    2019-07-14 01:00:00  99675.851562  0.000446  
                    2019-07-14 02:00:00  99737.945312  0.000877  
                    2019-07-14 03:00:00  99800.390625  0.001755  
                    2019-07-14 04:00:00  99801.429688  0.002193

In [6]:
# reset index
data_df = data_df.reset_index()
data_df.head()

,latitude,longitude,time,v10,t2m,ssr,sp,tp
0,18.299999,73.199997,2019-07-14 00:00:00,1.373911,297.332672,12508035.0,99645.671875,0.010906
1,18.299999,73.199997,2019-07-14 01:00:00,1.345363,297.305328,7625.0,99675.851562,0.000446
2,18.299999,73.199997,2019-07-14 02:00:00,1.217245,297.981140,161045.0,99737.945312,0.000877
3,18.299999,73.199997,2019-07-14 03:00:00,1.339882,297.890137,403873.0,99800.390625,0.001755
4,18.299999,73.199997,2019-07-14 04:00:00,1.227065,298.832306,1164014.0,99801.429688,0.002193


In [7]:
# find out the number of null values
data_df.isnull().sum()

latitude        0
longitude       0
time            0
v10          4752
t2m          4752
ssr          4752
sp           4752
tp           4752
dtype: int64

In [8]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88704 entries, 0 to 88703
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   latitude   88704 non-null  float64       
 1   longitude  88704 non-null  float64       
 2   time       88704 non-null  datetime64[ns]
 3   v10        83952 non-null  float32       
 4   t2m        83952 non-null  float32       
 5   ssr        83952 non-null  float32       
 6   sp         83952 non-null  float32       
 7   tp         83952 non-null  float32       
dtypes: datetime64[ns](1), float32(5), float64(2)
memory usage: 3.7 MB


In [9]:
# Deal with missing values with the nearest neighbor method
# fill the missing values with the nearest neighbor method
data_df = data_df.interpolate(method='ffill')
data_df.isnull().sum()


latitude     0
longitude    0
time         0
v10          0
t2m          0
ssr          0
sp           0
tp           0
dtype: int64

# Non-Linear Regression using Neural Network

In [10]:
time = data_df.time.values
time 


array(['2019-07-14T00:00:00.000000000', '2019-07-14T01:00:00.000000000',
       '2019-07-14T02:00:00.000000000', ...,
       '2021-07-24T21:00:00.000000000', '2021-07-24T22:00:00.000000000',
       '2021-07-24T23:00:00.000000000'], dtype='datetime64[ns]')

In [11]:
# drop the time column
data_df = data_df.drop(['time'], axis=1)
data_df.head()

,latitude,longitude,v10,t2m,ssr,sp,tp
0,18.299999,73.199997,1.373911,297.332672,12508035.0,99645.671875,0.010906
1,18.299999,73.199997,1.345363,297.305328,7625.0,99675.851562,0.000446
2,18.299999,73.199997,1.217245,297.981140,161045.0,99737.945312,0.000877
3,18.299999,73.199997,1.339882,297.890137,403873.0,99800.390625,0.001755
4,18.299999,73.199997,1.227065,298.832306,1164014.0,99801.429688,0.002193


## converting latitude and longitude to labels using LabelEncoder

In [12]:
columns = ['latitude', 'longitude']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in columns:
    data_df[i] = le.fit_transform(data_df[i])

## divide the data into x and y 

In [15]:
y = data_df['tp']
# get all columns except the target column
x = data_df.drop('tp', axis = 1)


In [16]:
y

0        0.010906
1        0.000446
2        0.000877
3        0.001755
4        0.002193
           ...   
88699    0.008408
88700    0.008755
88701    0.009243
88702    0.010010
88703    0.010651
Name: tp, Length: 88704, dtype: float32

In [17]:
x

,latitude,longitude,v10,t2m,ssr,sp
0,13,0,1.373911,297.332672,12508035.0,99645.671875
1,13,0,1.345363,297.305328,7625.0,99675.851562
2,13,0,1.217245,297.981140,161045.0,99737.945312
3,13,0,1.339882,297.890137,403873.0,99800.390625
4,13,0,1.227065,298.832306,1164014.0,99801.429688
...,...,...,...,...,...,...
88699,0,7,0.800230,294.726746,9399979.0,93469.609375
88700,0,7,0.838140,294.632019,9399979.0,93416.015625
88701,0,7,0.895234,294.614716,9399979.0,93378.546875
88702,0,7,0.786299,294.616455,9399979.0,93353.398438


## Linear Regression

In [18]:
# I run the linear regression
x2 = sm.add_constant(x) # necessary if we want regression equation to have intercept

mod_uni = sm.OLS(y,x2).fit()
y_pred_uni = mod_uni.fittedvalues

# Let's print out some model evaluation metrics!
# See your lessons for how to read this "summary"
print(mod_uni.summary())

                            OLS Regression Results                            
Dep. Variable:                     tp   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     2650.
Date:                Sun, 10 Apr 2022   Prob (F-statistic):               0.00
Time:                        03:01:17   Log-Likelihood:             2.2504e+05
No. Observations:               88704   AIC:                        -4.501e+05
Df Residuals:                   88697   BIC:                        -4.500e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2614      0.013     96.944      0.0

In [19]:
# Here is RMSE
rmse_err_uni = mean_squared_error(y, y_pred_uni,squared=False)
print('RMSE = ',rmse_err_uni)

# And R^2 (which is also outputted as a part of the above table)
print('R^2 = ',mod_uni.rsquared)

RMSE =  0.0191401513993705
R^2 =  0.1520001595495779


### Evaluation of the Linear Regression Model

## divide the data into train and test

In [37]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# convert all values to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')


## standardize the data


In [38]:
# standardize the data
scaler_x = StandardScaler()
x_train_scaled = scaler_x.fit_transform(x_train.values)
x_test_scaled = scaler_x.transform(x_test.values)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1,1))


In [39]:
y_test_scaled = scaler_y.fit_transform(y_test.values.reshape(-1,1))


## Convert the data to tensor

In [40]:
x_train_tensor = torch.from_numpy(x_train_scaled).requires_grad_(True)
x_test_tensor = torch.from_numpy(x_test_scaled).requires_grad_(True)
y_train_tensor = torch.from_numpy(y_train_scaled)

# Model parameters

In [41]:
batch_size = 1024
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)

## Model architecture

In [42]:
class NonLinearRegression(nn.Module):
    def __init__(self, input_size, hidden, output_size):
        super(NonLinearRegression, self).__init__()
        self.linear1 = nn.Linear(input_size, 60)
        self.linear2 = nn.Linear(60, 40)
        self.linear3 = nn.Linear(40, 20)
        self.linear4 = nn.Linear(20, hidden)
        self.linear5 = nn.Linear(hidden, output_size)
    def forward(self, x):
        x = F.tanh(self.linear1(x))
        x = F.tanh(self.linear2(x))
        x = F.tanh(self.linear3(x))
        x = F.tanh(self.linear4(x))
        x = self.linear5(x)
        return x.squeeze()

## define the model

In [43]:
input_dim = x_train_scaled.shape[1]
hidden_dim = 10
output_dim = 1
lr_model = NonLinearRegression(input_dim, hidden_dim, output_dim)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [44]:
lr_model.to(device)

criterion = F.mse_loss
optimizer = torch.optim.Adam(lr_model.parameters(), lr=3e-4)


## Fit model

In [45]:
def fit(num_epochs, model, criterion, opt):
    for epoch in range(num_epochs):
        for i, (x_batch, y_batch) in enumerate(train_loader):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            opt.zero_grad()
            loss.backward()
            opt.step()
            if (i+1) % 10 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                      .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

In [46]:
## Training the model
num_epochs = 25
fit(num_epochs, lr_model, criterion, optimizer)

Epoch [1/25], Step [10/70], Loss: 0.9711
Epoch [1/25], Step [20/70], Loss: 1.2075
Epoch [1/25], Step [30/70], Loss: 0.9961
Epoch [1/25], Step [40/70], Loss: 0.9454
Epoch [1/25], Step [50/70], Loss: 1.0797
Epoch [1/25], Step [60/70], Loss: 1.0514
Epoch [1/25], Step [70/70], Loss: 1.1517
Epoch [2/25], Step [10/70], Loss: 1.1099
Epoch [2/25], Step [20/70], Loss: 1.0557
Epoch [2/25], Step [30/70], Loss: 0.9498
Epoch [2/25], Step [40/70], Loss: 0.9701
Epoch [2/25], Step [50/70], Loss: 1.0218
Epoch [2/25], Step [60/70], Loss: 1.0312
Epoch [2/25], Step [70/70], Loss: 0.7911
Epoch [3/25], Step [10/70], Loss: 0.9182
Epoch [3/25], Step [20/70], Loss: 1.0505
Epoch [3/25], Step [30/70], Loss: 0.8548
Epoch [3/25], Step [40/70], Loss: 1.0654
Epoch [3/25], Step [50/70], Loss: 0.9795
Epoch [3/25], Step [60/70], Loss: 1.0825
Epoch [3/25], Step [70/70], Loss: 0.8943
Epoch [4/25], Step [10/70], Loss: 1.0837
Epoch [4/25], Step [20/70], Loss: 1.0391
Epoch [4/25], Step [30/70], Loss: 0.9846
Epoch [4/25], St

## Test model

In [51]:
y_test_tensor

tensor([[-0.8125],
        [-0.7479],
        [ 1.1547],
        ...,
        [-0.4562],
        [-0.6564],
        [-0.0832]])

In [47]:
y_test_tensor = torch.from_numpy(y_test_scaled)
y_pred = lr_model(x_test_tensor.to(device))


In [50]:
y_pred

tensor([0.0070, 0.0083, 0.0088,  ..., 0.0088, 0.0167, 0.0057],
       grad_fn=<SqueezeBackward0>)

## get r2 score


In [48]:
from sklearn.metrics import r2_score
print(f"The r2 score is {r2_score(y_test_scaled, y_pred.detach().numpy())}")

The r2 score is -0.001889908663518458


## save the model

In [49]:
# save the model
torch.save(lr_model.state_dict(), 'non_linear_regression_model.pt')